In [1]:
import cv2
import numpy as np

In [2]:
cv2.imshow("", np.random.random([100, 100, 3]))

In [27]:
import numpy as np
import os
from midiutil import MIDIFile
from midi2audio import FluidSynth
from pathlib import Path
from scipy.io import wavfile

def create_melody(sample_rate, bpm, duration_seconds, notes, output_folder):
    Path(output_folder).mkdir(parents=True, exist_ok=True)

    # Convert duration to beats
    seconds_per_beat = 60 / bpm
    duration_beats = duration_seconds / seconds_per_beat

    # Step 1: Create MIDI
    midi = MIDIFile(1)
    track = 0
    time = 0
    midi.addTempo(track, time, bpm)

    channel = 0
    volume = 100

    for i, note in enumerate(notes):
        start_time_beats = i * duration_beats
        midi.addNote(track, channel, note, start_time_beats, duration_beats, volume)

    midi_file = os.path.join(output_folder, "melody.mid")
    with open(midi_file, "wb") as output_file:
        midi.writeFile(output_file)

    # Step 2: Convert MIDI to audio
    soundfont = "/usr/share/sounds/sf2/FluidR3_GM.sf2"
    fs = FluidSynth(soundfont, sample_rate=sample_rate)
    wav_file = os.path.join(output_folder, "melody.wav")
    fs.midi_to_audio(midi_file, wav_file)

    # Step 3: Create note mapping array
    total_duration = duration_seconds * len(notes)
    n_samples = int(total_duration * sample_rate)
    note_array = np.zeros(n_samples, dtype=int)

    for i, note in enumerate(notes):
        start_sample = int(i * duration_seconds * sample_rate)
        end_sample = int((i + 1) * duration_seconds * sample_rate)
        note_array[start_sample:end_sample] = note

    note_array_file = os.path.join(output_folder, "melody_notes.npy")
    np.save(note_array_file, note_array)

    # Step 4: Clean-up WAV length
    wav_sample_rate, wav_data = wavfile.read(wav_file)

    expected_samples = n_samples
    if wav_data.shape[0] > expected_samples:
        wav_data = wav_data[:expected_samples]
        wavfile.write(wav_file, wav_sample_rate, wav_data)

    # Final check
    assert wav_data.shape[0] == len(note_array), (
        f"Length mismatch: WAV samples = {wav_data.shape[0]}, Note array = {len(note_array)}"
    )

    print(f"Done: saved to {output_folder}")

# Example usage:
create_melody(sample_rate=44100, bpm=120, duration_seconds=1, notes=[60,62], output_folder="../output_1")
create_melody(sample_rate=44100, bpm=120, duration_seconds=1, notes=[64,66], output_folder="../output_2")


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '../output_1/melody.wav'..
Done: saved to ../output_1
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '../output_2/melody.wav'..
Done: saved to ../output_2
